In [1]:
import os 
import csv
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
path = '../ce_data/output_512/dataset.csv'
df = pd.read_csv(path)
miss_ratios = pd.DataFrame()
for i in ["1024","2048","4096","8192"]:
    miss_ratios[i] = df['misses_'+i] / df['accesses_'+i]
    df = df.drop(columns=['misses_'+i, 'accesses_'+i, 'l1d_accesses_'+i])
df = df.drop(columns=["id"])
print(miss_ratios)
print(df)

          1024      2048      4096      8192
0     0.890827  0.795337  0.662562  0.123317
1     0.866846  0.765354  0.660919  0.077643
2     0.848553  0.727294  0.641544  0.262694
3     0.617509  0.223128  0.040222  0.001846
4     0.588169  0.200199  0.032464  0.001848
...        ...       ...       ...       ...
6186  0.785621  0.600917  0.436724  0.297468
6187  0.739754  0.549444  0.413295  0.301718
6188  0.934745  0.882877  0.810774  0.632815
6189  0.949653  0.904684  0.823035  0.558143
6190  0.998407  0.998407  0.998407  0.998407

[6191 rows x 4 columns]
             0         1         2        3        4        5        6  \
0     50816880  17323312  11868112  7701840  3853504  5287776  7547888   
1     17361520   6023600   4203744  2591488  1828656  1847648  2429008   
2     58283520  17249296  14592592  9466576  7204896  5614336  8940048   
3      3098304   3037808   2698720  2692176  1999568  1238496   939408   
4      1623056   1618144   1501200  1353728  1014736   670656   5

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,miss_ratios, test_size=0.3)

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError
# Create the model
model = Sequential()

# Add the input layer and the first hidden layer
model.add(Dense(1024, input_dim=896, activation='sigmoid'))

# Add the second hidden layer
model.add(Dense(1024, activation='sigmoid'))

# Add the output layer
model.add(Dense(4, activation='sigmoid'))

# Compile the model
model.compile(loss=MeanSquaredError() , optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              918528    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 1,972,228
Trainable params: 1,972,228
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=10)

Epoch 1/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0686
Epoch 2/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0525
Epoch 3/10
136/136 [==============================] - 1s 11ms/step - loss: 0.0359
Epoch 4/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0327
Epoch 5/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0289
Epoch 6/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0277
Epoch 7/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0270
Epoch 8/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0248
Epoch 9/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0268
Epoch 10/10
136/136 [==============================] - 2s 11ms/step - loss: 0.0240


In [17]:
f = model(X_test.to_numpy())

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Compare columns
for i,j in enumerate(['1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = y_test.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 1024	: 0.035968517155317
Mean squared error for cache size 2048	: 0.028481030381368375
Mean squared error for cache size 4096	: 0.023978277399955557
Mean squared error for cache size 8192	: 0.02555132078879714
